In [1]:
import torch
import numpy as np
from PIL import Image, ImageSequence

from diffusion_wrapper import DiffusionWrapper
from heatmap_generator import HeatmapGenerator
from zero_shot_tracker import ZeroShotTracker


diffusion_wrapper = DiffusionWrapper('../text-to-video-ms-1.7b/')
heatmap_generator = HeatmapGenerator()
zero_shot_tracker = ZeroShotTracker()

/home/max/miniconda3/envs/diffusers/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading pipeline components...:  20%|██        | 1/5 [00:00<00:01,  3.04it/s]2024-05-19 17:36:02.215514: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-19 17:36:02.252138: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-19 17:36:02.958481:

In [2]:
from evaluation_datasets import create_davis_dataset

davis_dataset = create_davis_dataset('../tapvid_davis/tapvid_davis.pkl')

first_video = {}

for data in davis_dataset:
    first_video = data['davis']
    break

video_tensor = torch.tensor(first_video['video'])

In [3]:
video_features_dict = diffusion_wrapper.extract_video_features(video_tensor, "")

/home/max/miniconda3/envs/diffusers/lib/python3.11/site-packages/torch/nn/modules/conv.py:605: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv3d(
/home/max/miniconda3/envs/diffusers/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


In [4]:
video_features = diffusion_wrapper.concatenate_video_features({'up_block': video_features_dict['up_block'][2:3]})

In [5]:
video_features = video_features.permute(0, 2, 3, 1).float()
heatmaps = heatmap_generator.generate(video_features, (93, 137, 0))
tracks = zero_shot_tracker.track(heatmaps)

In [6]:
zero_shot_tracker.place_marker_in_frames(first_video['video'].squeeze(), tracks)
heatmap_generator.safe_heatmap_as_gif(heatmaps)